In [8]:
import import_ipynb
from Functions import *

# Read Frames as Tensors & Add Inconsistencies

In [9]:
Frames = get_video_frames("VDB",13,(200,200))
for _ in range(3):
    Frames[1] = AddOneInc1(Frames[1],Op=10,x=50,y=20,l=20)
    Frames[2] = AddOneInc1(Frames[2],Op=10,x=50,y=20,l=20)
    Frames[5] = AddOneInc1(Frames[4],Op=6,x=10,y=80,l=30)
    Frames[6] = AddOneInc1(Frames[5],Op=6,x=10,y=80,l=30)
    Frames[9] = AddOneInc1(Frames[9],x=100,y=100,l=40)
    Frames[10] = AddOneInc1(Frames[10],x=100,y=100,l=40)

# To Numpy

In [60]:
ThroughFrames([f.permute(1,2,0).numpy() for f in Frames])

# Calculate Optical Flow

In [61]:
Optflow = [OF_tensor1(Frames[i],Frames[i+1],kernel_size=11) for i in range(len(Frames)-1)]

KeyboardInterrupt: 

# Draw Optical Flow

In [62]:
Flow = [draw_optical_flow1(Frames[i],Optflow[i],step=11,scale=100.0) for i in range(len(Optflow))]

# Display Flow

In [13]:
#################    1     ########

In [63]:
ThroughFrames(Flow)

# Differences in Optical Flow

In [15]:
dflow = [Optflow[i+1]-Optflow[i] for i in range(len(Optflow)-1)]

# Draw Optical Flow Differences

In [64]:
DifFlow = [draw_optical_flow1(Frames[i],dflow[i],step=11,scale=100.0) for i in range(len(dflow))]

# Display Differential OF

In [17]:
##############        1        ########

In [65]:
ThroughFrames(DifFlow)

# Frames Differences to Numpy

In [67]:
Diff = [Frames[i]-Frames[i+1] for i in range(len(Frames)-1)]
Diff = [d.permute(1,2,0).numpy() for d in Diff]
Diff = [((d-np.min(d))/(np.max(d)-np.min(d))*255).clip(0,255).astype(np.uint8) for d in Diff]

# Display Frames Differences

In [68]:
##########            3        ##########

In [69]:
ThroughFrames(Diff)

# Frames Differences to Tensor

In [22]:
Diff = [torch.from_numpy(d).permute(2,0,1) for d in Diff]

# Draw Flow in Frames Differences

In [23]:
FlowDiff = [draw_optical_flow1(Diff[i],OF_tensor1(Diff[i],Diff[i+1]),step=10,scale=1.0) for i in range(len(Diff)-1)]

# Display Flow in Frame Differences

In [24]:
############           1     #########

In [70]:
ThroughFrames(FlowDiff)

# Calculate Frequencies

In [26]:
Freqs = [torch.fft.fft2(f) for f in Frames]

# Display Frequencies

In [27]:
DFreqs = [display_frequency_image(f) for f in Freqs]

In [71]:
ThroughFrames(DFreqs)

# Calculate Differences in Frequencies

In [29]:
DifFreqs = [Freqs[i]-Freqs[i+1] for i in range(len(Freqs)-1)]

# Display Differences in Frequencies

In [30]:
DDiffFreqs = [display_frequency_image(d) for d in DifFreqs]

In [31]:
###########      2      ############

In [72]:
ThroughFrames(DDiffFreqs)

# Calculate Frequencies in Pixel Differences

In [33]:
PixDifFreqs = [torch.fft.fft2(d) for d in Diff]

# Display Frequencies in Pixel Differences

In [34]:
DDPix = [display_frequency_image(d) for d in PixDifFreqs]

In [35]:
#############           2           ###########

In [73]:
ThroughFrames(DDPix)

# Edges

In [37]:
Edges = [Frame2Numpy(Edges_tensor(f)) for f in Frames]

In [38]:
###########         1          #############

In [74]:
ThroughFrames(Edges)

# Edges Differences

In [40]:
EdDif = [Edges[i]-Edges[i+1] for i in range(len(Edges)-1)]

In [41]:
##############            3          #############

In [75]:
ThroughFrames(EdDif)

# Edges Flow

In [43]:
EdgesT = [Frame2Torch(e).squeeze(0) for e in Edges]
EdOF = [OF_tensor1(EdgesT[i],EdgesT[i+1],kernel_size=11) for i in range(len(EdgesT)-1)]

In [44]:
DrawEdgOF = [draw_optical_flow1(Frames[i],EdOF[i],step=11,scale=20.0) for i in range(len(EdOF))]

In [45]:
#########               2        ########

In [76]:
ThroughFrames(DrawEdgOF)

# SSIM

In [47]:
def create_gaussian_window(window_size, channels, sigma):
    # Create a 1D Gaussian kernel
    x = torch.arange(window_size, dtype=torch.float32) - (window_size - 1) / 2
    gauss = torch.exp(-(x**2) / (2 * sigma**2))
    gauss /= gauss.sum()  # Normalize the 1D kernel to ensure the sum is 1

    # Create a 2D Gaussian kernel by taking the outer product
    gauss_2d = gauss[:, None] * gauss[None, :]

    # Reshape the kernel to be compatible with conv2d
    kernel = gauss_2d.expand(channels, 1, window_size, window_size)

    return kernel

In [48]:
def TSSIM(img1, img2, window_size=11, sigma=1.5, C1=0.01**2, C2=0.03**2):

    channels = img1.size(1)
    gaussian_window = create_gaussian_window(window_size, channels, sigma).to(img1.device)

    # Compute local means using Gaussian filter
    mu1 = Fun.conv2d(img1, gaussian_window, padding=window_size // 2, groups=channels)
    mu2 = Fun.conv2d(img2, gaussian_window, padding=window_size // 2, groups=channels)

    # Compute local variances and covariance
    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2

    sigma1_sq = Fun.conv2d(img1 * img1, gaussian_window, padding=window_size // 2, groups=channels) - mu1_sq
    sigma2_sq = Fun.conv2d(img2 * img2, gaussian_window, padding=window_size // 2, groups=channels) - mu2_sq
    sigma12 = Fun.conv2d(img1 * img2, gaussian_window, padding=window_size // 2, groups=channels) - mu1_mu2

    # Compute SSIM map
    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / (
        (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
    )

    mean_ssim = ssim_map.mean()  # Compute the mean SSIM value

    return mean_ssim, ssim_map  # Return both mean SSIM and SSIM map

In [49]:
TScores = [TSSIM(Frames[i].unsqueeze(0),Frames[i+1].unsqueeze(0)) for i in range(len(Frames)-1)]
TScore = [t[0].item() for t in TScores]
TPixScores = [t[1] for t in TScores]

In [50]:
TScore

[0.8996657133102417,
 0.9880629181861877,
 0.9062681198120117,
 0.9278402328491211,
 0.9978240728378296,
 0.9976814985275269,
 0.8129255175590515,
 0.9855190515518188,
 0.8816332221031189,
 0.957511305809021,
 0.9729323983192444,
 0.8441409468650818]

In [51]:
TPixImg = [Frame2Numpy(t) for t in TPixScores]

In [52]:
#########            3             ###########3

In [77]:
ThroughFrames(TPixImg)

In [54]:
TPixImg[0].shape

(200, 200, 3)

# Object Detection

In [55]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as TrFun

model = fasterrcnn_resnet50_fpn(pretrained=True).eval()

COCO_LABELS = {
    1: "Person", 2: "Bicycle", 3: "Car", 4: "Motorcycle", 5: "Airplane", 6: "Bus", 7: "Train", 
    8: "Truck", 9: "Boat", 10: "Traffic Light", 11: "Fire Hydrant", 13: "Stop Sign", 14: "Parking Meter", 
    15: "Bench", 16: "Bird", 17: "Cat", 18: "Dog", 19: "Horse", 20: "Sheep", 21: "Cow", 22: "Elephant", 
    23: "Bear", 24: "Zebra", 25: "Giraffe", 27: "Backpack", 28: "Umbrella", 31: "Handbag", 32: "Tie", 
    33: "Suitcase", 34: "Frisbee", 35: "Skis", 36: "Snowboard", 37: "Sports Ball", 38: "Kite", 
    39: "Baseball Bat", 40: "Baseball Glove", 41: "Skateboard", 42: "Surfboard", 43: "Tennis Racket", 
    44: "Bottle", 46: "Wine Glass", 47: "Cup", 48: "Fork", 49: "Knife", 50: "Spoon", 51: "Bowl", 
    52: "Banana", 53: "Apple", 54: "Sandwich", 55: "Orange", 56: "Broccoli", 57: "Carrot", 58: "Hot Dog", 
    59: "Pizza", 60: "Donut", 61: "Cake", 62: "Chair", 63: "Couch", 64: "Potted Plant", 65: "Bed", 
    67: "Dining Table", 70: "Toilet", 72: "TV", 73: "Laptop", 74: "Mouse", 75: "Remote", 76: "Keyboard", 
    77: "Cell Phone", 78: "Microwave", 79: "Oven", 80: "Toaster", 81: "Sink", 82: "Refrigerator", 
    84: "Book", 85: "Clock", 86: "Vase", 87: "Scissors", 88: "Teddy Bear", 89: "Hair Drier", 90: "Toothbrush"
}

def detect_objects(frame):
    frame_tensor = TrFun.to_tensor(frame).unsqueeze(0)
    predictions = model(frame_tensor)
    return predictions[0]['boxes'], predictions[0]['labels'],predictions[0]['scores']

In [56]:
Boxes,Labels,Prob = detect_objects(Frame2Numpy(Frames[8]))

In [57]:
Boxes[torch.nonzero(Prob>0.5)],Labels[torch.nonzero(Prob>0.5)]

(tensor([[[109.7835,  17.1182, 197.9163, 193.3935]],
 
         [[ 49.7932,  41.6467, 125.4709, 157.3019]],
 
         [[124.8270,  50.6015, 172.8132,  84.5231]],
 
         [[ 65.8218,  25.0697, 178.8323, 185.6025]]], grad_fn=<IndexBackward0>),
 tensor([[ 1],
         [ 1],
         [43],
         [ 1]]))

In [58]:
def Draw_objects(Frames):
    Draw = [cv2.cvtColor(Frame2Numpy(f.clone()),cv2.COLOR_BGR2RGB) for f in Frames]
    for idx, d in enumerate(Draw):
        Boxes, Labels, Prob = detect_objects(d)
        mask = Prob > 0.75
        Boxes = Boxes[mask]
        Labels = Labels[mask]
        for b, l in zip(Boxes, Labels):
            x1, y1, x2, y2 = int(b[0].item()), int(b[1].item()), int(b[2].item()), int(b[3].item())
            label_id = int(l.item())
            label_name = COCO_LABELS.get(label_id, f"Unknown ({label_id})")
            label_text = f"{label_name}"
            cv2.rectangle(d, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
            text_x = x1
            text_y = y1 - 10 if y1 - 10 > 10 else y1 + 10
            cv2.rectangle(d, (text_x, text_y - text_size[1] - 4), (text_x + text_size[0], text_y), (0, 255, 0), -1)
            cv2.putText(d, label_text, (text_x, text_y - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    return [cv2.cvtColor(d,cv2.COLOR_RGB2BGR) for d in Draw]

In [59]:
DetectObjImg = Draw_objects(Frames)

KeyboardInterrupt: 

In [ ]:
########         1        ########

In [ ]:
ThroughFrames(DetectObjImg)

# Entropy

In [78]:
def entropy_map(image_tensor: torch.Tensor, window_size: int) -> torch.Tensor:
    assert image_tensor.ndim == 3 and image_tensor.shape[0] == 3, \
        "Input tensor must have shape (3, H, W)."
    padding = window_size // 2
    padded_tensor = torch.nn.functional.pad(image_tensor, (padding, padding, padding, padding), mode='reflect')
    def compute_entropy(channel):
        entropy_map = torch.zeros_like(channel)
        for i in range(channel.shape[0]):
            for j in range(channel.shape[1]):
                window = padded_tensor[0, i:i + window_size, j:j + window_size]
                hist = torch.histc(window, bins=256, min=0, max=1)
                hist = hist / hist.sum()
                entropy = -torch.sum(hist * torch.log2(hist + 1e-12))
                entropy_map[i, j] = entropy
        return entropy_map
    entropy_maps = torch.stack([compute_entropy(image_tensor[c]) for c in range(3)])
    return entropy_maps

In [79]:
Entropies = []
for f in Frames:
    entropy_maps = entropy_map(f,10)
    Entropy = entropy_maps.permute(1,2,0).numpy()
    Entropy1 = cv2.normalize(Entropy, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    Entropies.append(Entropy1)

In [80]:
#########       1        ############

In [81]:
ThroughFrames(Entropies)